<a href="https://colab.research.google.com/github/athipan1/Database_painaidee/blob/main/tests/test_all_apis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧪 ทดลอง API ทั้งหมดใน Google Colab

## Database Painaidee - API Testing Notebook

**ทดสอบ API ของระบบจัดการข้อมูลสถานที่ท่องเที่ยวไทย**

This notebook demonstrates how to:
- 🚀 Deploy the complete Thai tourism API temporarily
- 🧪 Test all API endpoints with real examples
- 🗣️ Try the `/api/talk` conversational AI feature
- 🎤 Test voice input/output functionality (if supported)
- 📊 View real-time dashboard and analytics

**Features to test:**
- Tourism attraction search and management
- AI-powered keyword extraction and recommendations
- Behavior analytics and user tracking
- Real-time dashboard with statistics
- Conversational AI for tourism queries

## 🔧 Setup and Installation

**Step 1: Clone the repository and install dependencies**

In [ ]:
# Clone the Database Painaidee repository
!git clone https://github.com/athipan1/Database_painaidee.git
%cd Database_painaidee

# Install required packages
!pip install -r requirements.txt

# Install additional packages for Colab environment
!pip install pyngrok flask-cors

print("✅ Repository cloned and dependencies installed!")

## 🗃️ Database Setup

**Step 2: Setup PostgreSQL and Redis (using SQLite for simplicity in Colab)**

In [ ]:
import os

# Create environment configuration for Colab
env_config = '''
FLASK_ENV=development
SECRET_KEY=colab-test-secret-key-for-demo
DATABASE_URL=sqlite:///painaidee_test.db
REDIS_URL=redis://localhost:6379/0
CELERY_BROKER_URL=redis://localhost:6379/0
DEBUG=True
TESTING=True
PAGINATION_ENABLED=true
PAGINATION_PAGE_SIZE=10
'''

# Write .env file
with open('.env', 'w') as f:
    f.write(env_config)

# Install and start Redis (lightweight for Colab)
!apt-get update && apt-get install -y redis-server
!redis-server --daemonize yes

print("✅ Database and Redis configured for Colab!")

## 🌐 Public Access Setup

**Step 3: Setup ngrok for public API access**

In [ ]:
import os
from pyngrok import ngrok

# Optional: Set your ngrok auth token for longer sessions
# ngrok.set_auth_token("YOUR_NGROK_TOKEN")

# Initialize database
!python init_db.py

print("✅ Database initialized!")
print("🚀 Ready to start the API server...")

## 🚀 Start the API Server

**Step 4: Launch the Flask API with public access**

In [ ]:
import threading
import time
from flask import Flask
from app import create_app

# Create Flask app
app = create_app()

# Start Flask server in background thread
def run_flask():
    app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)

# Start server in background
flask_thread = threading.Thread(target=run_flask, daemon=True)
flask_thread.start()

# Wait for server to start
time.sleep(3)

# Create ngrok tunnel
public_url = ngrok.connect(5000)

print(f"🌐 API is now publicly accessible at: {public_url}")
print(f"📊 Dashboard: {public_url}/api/dashboard/")
print(f"🏛️ Attractions API: {public_url}/api/attractions")
print(f"🗣️ Conversational AI: {public_url}/api/talk")
print(f"❤️ Health Check: {public_url}/api/health")
print("\n✅ API server is running and ready for testing!")

# Store the URL for later use
API_BASE_URL = str(public_url)

## 📊 Load Sample Tourism Data

**Step 5: Populate the database with Thai tourism data**

In [ ]:
import requests
import json

# Sync data from Tourism Authority of Thailand (TAT)
print("📊 Loading Thai tourism data...")

try:
    # Try to sync TAT data first
    response = requests.post(f"{API_BASE_URL}/api/attractions/sync/tat", timeout=30)
    if response.status_code == 200:
        result = response.json()
        print(f"✅ TAT Data loaded: {result.get('message', 'Success')}")
    else:
        print("⚠️ TAT data not available, loading test data...")
        # Fallback to test data
        response = requests.post(f"{API_BASE_URL}/api/attractions/sync", timeout=30)
        if response.status_code == 200:
            result = response.json()
            print(f"✅ Test data loaded: {result.get('message', 'Success')}")
        
except Exception as e:
    print(f"❌ Error loading data: {e}")
    print("Adding sample data manually...")
    
    # Add sample attraction manually
    sample_data = {
        "title": "วัดพระแก้ว กรุงเทพมหานคร",
        "body": "วัดพระแก้วเป็นวัดที่สำคัญที่สุดในประเทศไทย ตั้งอยู่ในพระบรมมหาราชวัง กรุงเทพฯ เป็นที่ประดิษฐานพระพุทธมหามณีรัตนปฏิมากร หรือพระแก้วมรกต",
        "userId": 1
    }
    
    try:
        response = requests.post(f"{API_BASE_URL}/api/attractions", json=sample_data)
        if response.status_code == 201:
            print("✅ Sample attraction added successfully!")
    except Exception as e:
        print(f"⚠️ Could not add sample data: {e}")

print("\n🎉 Database is ready for testing!")

## 🧪 API Testing Examples

**Step 6: Test all API endpoints**

### 1. Health Check

In [ ]:
# Test health check endpoint
response = requests.get(f"{API_BASE_URL}/api/health")
print(f"Health Check Status: {response.status_code}")
print(f"Response: {response.json()}")

### 2. Attractions API

In [ ]:
# Get all attractions
response = requests.get(f"{API_BASE_URL}/api/attractions")
attractions = response.json()

print(f"📊 Total attractions: {len(attractions.get('data', []))}")
print("\n🏛️ Sample attractions:")

for i, attraction in enumerate(attractions.get('data', [])[:3]):
    print(f"\n{i+1}. {attraction.get('title', 'No title')}")
    print(f"   📍 Location: {attraction.get('province', 'Unknown')}")
    print(f"   📝 Description: {attraction.get('body', 'No description')[:100]}...")
    if attraction.get('latitude') and attraction.get('longitude'):
        print(f"   🗺️ Coordinates: {attraction['latitude']}, {attraction['longitude']}")

### 3. Dashboard Statistics

In [ ]:
# Get dashboard statistics
response = requests.get(f"{API_BASE_URL}/api/dashboard/stats")
stats = response.json()

print("📊 Dashboard Statistics:")
print(f"   Total Attractions: {stats.get('total_attractions', 0)}")
print(f"   Geocoded Attractions: {stats.get('geocoded_attractions', 0)}")
print(f"   Recent Syncs: {stats.get('recent_syncs', 0)}")
print(f"   System Health: {stats.get('system_health', 'Unknown')}")

print(f"\n🌐 Visit the interactive dashboard: {API_BASE_URL}/api/dashboard/")

### 4. 🗣️ Conversational AI - `/api/talk` Feature

**Test the conversational AI for tourism queries**

In [ ]:
# Test conversational AI endpoint
print("🗣️ Testing Conversational AI (/api/talk)")
print("=" * 50)

# Sample queries to test
test_queries = [
    "แนะนำสถานที่ท่องเที่ยวในกรุงเทพฯ",
    "วัดไหนสวยที่สุดในไทย",
    "What are the best temples to visit in Bangkok?",
    "ช่วยหาข้อมูลเกี่ยวกับวัดพระแก้ว"
]

for i, query in enumerate(test_queries, 1):
    print(f"\n{i}. Query: {query}")
    
    try:
        # Send request to talk API
        talk_data = {
            "message": query,
            "user_id": "colab_test_user"
        }
        
        response = requests.post(f"{API_BASE_URL}/api/talk", json=talk_data, timeout=15)
        
        if response.status_code == 200:
            result = response.json()
            print(f"   ✅ Response: {result.get('response', 'No response')}")
            
            # Show related attractions if available
            if result.get('attractions'):
                print(f"   🏛️ Related attractions: {len(result['attractions'])}")
                for attraction in result['attractions'][:2]:
                    print(f"      - {attraction.get('title', 'Unknown')}")
        else:
            print(f"   ❌ Error: {response.status_code} - {response.text}")
            
    except Exception as e:
        print(f"   ⚠️ Error testing query: {e}")

print("\n🎉 Conversational AI testing completed!")

### 5. 📋 Complete Sample Request & Response

**Detailed example of API usage**

In [ ]:
import json

print("📋 Complete API Request & Response Example")
print("=" * 60)

# Sample request data
sample_request = {
    "message": "แนะนำวัดสวยๆ ในกรุงเทพ พร้อมข้อมูลการเดินทาง",
    "user_id": "demo_user_123",
    "preferences": {
        "language": "th",
        "location": "Bangkok",
        "interest": "temples"
    }
}

print("📤 Sample Request:")
print(f"POST {API_BASE_URL}/api/talk")
print("Content-Type: application/json")
print("\nRequest Body:")
print(json.dumps(sample_request, indent=2, ensure_ascii=False))

print("\n" + "="*60)

# Send the request
try:
    response = requests.post(
        f"{API_BASE_URL}/api/talk", 
        json=sample_request,
        headers={'Content-Type': 'application/json'},
        timeout=20
    )
    
    print("📥 Sample Response:")
    print(f"Status Code: {response.status_code}")
    print("\nResponse Body:")
    
    if response.status_code == 200:
        response_data = response.json()
        print(json.dumps(response_data, indent=2, ensure_ascii=False))
        
        # Extract key information
        print("\n" + "="*60)
        print("📊 Response Summary:")
        print(f"✅ AI Response: {response_data.get('response', 'No response')}")
        print(f"🏛️ Attractions Found: {len(response_data.get('attractions', []))}")
        print(f"🕒 Processing Time: {response_data.get('processing_time', 'N/A')}")
        print(f"💬 Conversation ID: {response_data.get('conversation_id', 'N/A')}")
        
    else:
        print(response.text)
        
except Exception as e:
    print(f"❌ Error: {e}")

print("\n🎉 Sample request/response demonstration completed!")

### 6. 🎤 Voice Input/Output Testing (Optional)

**Test voice functionality if supported**

In [ ]:
# Note: Voice functionality may require additional setup in Colab
print("🎤 Voice Input/Output Testing")
print("=" * 40)

# Check if voice endpoints exist
voice_endpoints = [
    "/api/voice/speech-to-text",
    "/api/voice/text-to-speech",
    "/api/talk/voice"
]

print("🔍 Checking available voice endpoints...")

for endpoint in voice_endpoints:
    try:
        response = requests.get(f"{API_BASE_URL}{endpoint}")
        if response.status_code != 404:
            print(f"   ✅ {endpoint} - Available (Status: {response.status_code})")
        else:
            print(f"   ❌ {endpoint} - Not found")
    except:
        print(f"   ⚠️ {endpoint} - Connection error")

# Text-to-speech simulation (if API supports it)
print("\n🔊 Simulating text-to-speech functionality:")
sample_text = "สวัสดีครับ ยินดีต้อนรับสู่ระบบข้อมูลสถานที่ท่องเที่ยวไทย"
print(f"Text to convert: {sample_text}")
print("Note: Voice features may require additional browser permissions and setup.")
print("For full voice functionality, consider using the web interface directly.")

print("\n✅ Voice testing section completed!")

### 7. 🤖 AI Features Testing

**Test AI-powered features**

In [ ]:
print("🤖 Testing AI Features")
print("=" * 30)

# Test keyword extraction
print("\n1. 🔍 Keyword Extraction:")
try:
    keyword_data = {
        "text": "วัดพระแก้วเป็นวัดที่สำคัญที่สุดในประเทศไทย ตั้งอยู่ในพระบรมมหาราชวัง กรุงเทพฯ เป็นที่ประดิษฐานพระแก้วมรกต",
        "language": "th"
    }
    
    response = requests.post(f"{API_BASE_URL}/api/ai/keywords/extract", json=keyword_data)
    if response.status_code == 200:
        result = response.json()
        print(f"   ✅ Keywords extracted: {result.get('keywords', [])}")
    else:
        print(f"   ⚠️ Keyword extraction unavailable (Status: {response.status_code})")
except Exception as e:
    print(f"   ❌ Error: {e}")

# Test recommendations
print("\n2. 💡 AI Recommendations:")
try:
    response = requests.get(f"{API_BASE_URL}/api/ai/recommendations/demo_user")
    if response.status_code == 200:
        result = response.json()
        recommendations = result.get('recommendations', [])
        print(f"   ✅ Recommendations found: {len(recommendations)}")
        for i, rec in enumerate(recommendations[:3], 1):
            print(f"      {i}. {rec.get('title', 'Unknown')} (Score: {rec.get('score', 0):.2f})")
    else:
        print(f"   ⚠️ Recommendations unavailable (Status: {response.status_code})")
except Exception as e:
    print(f"   ❌ Error: {e}")

# Test behavior tracking
print("\n3. 📊 Behavior Analytics:")
try:
    behavior_data = {
        "user_id": "colab_test_user",
        "action": "view_attraction",
        "attraction_id": 1,
        "timestamp": "2024-01-01T12:00:00Z"
    }
    
    response = requests.post(f"{API_BASE_URL}/api/behavior/track", json=behavior_data)
    if response.status_code == 200:
        print("   ✅ User behavior tracked successfully")
        
        # Get behavior analysis
        response = requests.get(f"{API_BASE_URL}/api/behavior/analyze/colab_test_user")
        if response.status_code == 200:
            analysis = response.json()
            print(f"   📈 Behavior patterns: {len(analysis.get('patterns', []))}")
    else:
        print(f"   ⚠️ Behavior tracking unavailable (Status: {response.status_code})")
except Exception as e:
    print(f"   ❌ Error: {e}")

print("\n🎉 AI features testing completed!")

## 🎉 Testing Summary

**Your API is now running and accessible!**

In [ ]:
print("🎉 Database Painaidee API Testing Summary")
print("=" * 50)
print(f"🌐 Public API URL: {API_BASE_URL}")
print(f"📊 Dashboard: {API_BASE_URL}/api/dashboard/")
print(f"🏛️ Attractions: {API_BASE_URL}/api/attractions")
print(f"🗣️ Talk API: {API_BASE_URL}/api/talk")
print(f"❤️ Health: {API_BASE_URL}/api/health")

print("\n✅ Features Tested:")
print("   🏛️ Tourism attractions database")
print("   🗣️ Conversational AI (/api/talk)")
print("   🤖 AI keyword extraction and recommendations")
print("   📊 Behavior analytics and tracking")
print("   📈 Real-time dashboard")
print("   🎤 Voice capabilities (if supported)")

print("\n🔧 Next Steps:")
print("   1. Visit the dashboard for interactive exploration")
print("   2. Try more API endpoints with different queries")
print("   3. Test the conversational AI with Thai/English questions")
print("   4. Explore the complete API documentation")

print("\n💡 Tips:")
print("   - API will be available until this Colab session ends")
print("   - Use the dashboard for a visual interface")
print("   - Try asking tourism questions in Thai or English")
print("   - Check /api/ for complete endpoint documentation")

print("\n🙏 Thank you for testing Database Painaidee!")
print("   Repository: https://github.com/athipan1/Database_painaidee")

## 📚 Additional Resources

- **GitHub Repository**: [athipan1/Database_painaidee](https://github.com/athipan1/Database_painaidee)
- **Full Documentation**: Available in the repository README
- **API Endpoints**: Visit `{your-ngrok-url}/api/` for complete documentation
- **Dashboard**: Interactive web interface at `{your-ngrok-url}/api/dashboard/`

### API Endpoints Summary:
- `GET /api/health` - Health check
- `GET /api/attractions` - List all attractions
- `POST /api/talk` - Conversational AI
- `GET /api/dashboard/` - Web dashboard
- `POST /api/ai/keywords/extract` - Keyword extraction
- `GET /api/ai/recommendations/{user_id}` - AI recommendations
- `POST /api/behavior/track` - Behavior tracking

**Note**: This is a temporary deployment for testing. For permanent deployment, consider using Vercel, Hugging Face Spaces, or your own server infrastructure.